In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from transformers import BertTokenizer, BertModel
import torch.optim.lr_scheduler as lr_scheduler
from pytorch_model_summary import summary
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch import optim
import torch.nn as nn
import torch

import numpy as np
import unicodedata
import random
import math
import re

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import requests
# import beautifulsoup4 as bs4
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_encoder = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
SOS_token = 101
EOS_token = 102

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z'/!?]+", r" ", s)
    s = re.sub(r"'", " '", s)
    return s.strip()

In [ ]:
def readLangs():
    print("Reading lines...")

    lines = []
    counter = 0

    with open("/media/nathanmon/389E28739E282BB6/Users/Natha/Datasets/MyJarvisConversation/conversation.txt", "r") as f:
        for line in f.readlines():
            if line[0] == "U":
                lines.append("")
                lines[counter] += line[6:] + "/t"
            elif line[0] == "J":
                line = line.replace("/u", "/u ")
                lines[counter] += line[8:]
                counter += 1

    return lines

In [ ]:
MAX_LENGTH = 30

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData():
    pairs = readLangs()
    pairs = filterPairs(pairs)
    for i, pair in enumerate(pairs):
        pairs[i] = pair.lower().replace("\n", "").split("/t")
    return pairs

In [ ]:
keywords = ["/u shopping", "/u todolist", "/u wiki", "/u volume", "/a/"]
filenames = ["shopping_items", "todo_list_items", "wiki_queries", "volumes", "apps"]
augments = {"shopping_items": [], "todo_list_items": [],
            "wiki_queries": [], "volumes": [], "apps": []}

for keyword, filename in zip(keywords, filenames):
    with open(f"/media/nathanmon/389E28739E282BB6/Users/Natha/Datasets/MyJarvisConversation/{filename}.txt", "r") as f:
        for line in f.readlines():
            augments[filename].append(line.replace("\n", "").strip())
            
numbers = ["zero", "one", "two", "three", "four", 
   "five", "six", "seven", "eight", "nine",
   "ten", "eleven", "twelve", "thirteen",
   "fourteen", "fifteen", "sixteen", 
   "seventeen", "eighteen", "nineteen",
   "twenty", "thirty", "forty", "fifty",
   "sixty", "seventy", "eighty", "ninety",
   "hundred", "thousand", "million", "billion",
   "trillion", "quadrillion", "quintillion", "mute", "?"]

In [ ]:
# def sentence2num(sentence):
#     words = sentence.split(" ")
#     filtered = []
#     for word in words:
#         if word.lower() in numbers:
#             filtered.append(word)

#     return " ".join(filtered)

# def find_tgt(response, loc="'"):
#     lower = response.index(loc) + len(loc)
#     upper = response[lower:].index(loc) + lower
#     return response[lower:upper]

# def augment(inp, tgt):
#     keywords = ["/u shopping", "/u todolist", "/u wiki", "/u volume", "/a"]
#     filenames = ["shopping_items", "todo_list_items", "wiki_queries", "volumes", "apps"]

#     for keyword, filename in zip(keywords, filenames):
#         if keyword in tgt or keyword in inp:
#             if keyword == "/u volume":
#                 prev_item = sentence2num(find_tgt(tgt))
#             elif keyword == "/a":
#                 prev_item = find_tgt(inp, "/a")
#             else:
#                 prev_item = find_tgt(tgt)

#             if keyword != "/uvolume" or (prev_item != "?"and prev_item != "Mute"):
#                 replacement = random.choice(augments[filename])
#                 inp = inp.replace(prev_item, replacement)
#                 if keyword == "/a":
#                     prev_item = find_tgt(tgt)

#                 tgt = tgt.replace(prev_item, replacement)
#                 return inp.replace("/a", ""), tgt.replace("/a", "")
#     return inp, tgt

# def get_dataloader(batch_size):
#     pairs = prepareData()
#     train_pairs, val_pairs = pairs[:225], pairs[225:]

#     train_n = len(train_pairs)
#     val_n = len(val_pairs)
#     train_input_ids = np.zeros((train_n, MAX_LENGTH), dtype=np.int32)
#     train_target_ids = np.zeros((train_n, MAX_LENGTH), dtype=np.int32)
#     val_input_ids = np.zeros((val_n, MAX_LENGTH), dtype=np.int32)
#     val_target_ids = np.zeros((val_n, MAX_LENGTH), dtype=np.int32)

#     for idx, (inp, tgt) in enumerate(train_pairs):
#         inp, tgt = augment(inp, tgt)
#         inp_encoded = tokenizer.encode(inp)
#         tgt_ids = tokenizer.encode(tgt)
#         train_input_ids[idx, :len(inp_encoded)] = inp_encoded
#         train_target_ids[idx, :len(tgt_ids)] = tgt_ids
        
#     for idx, (inp, tgt) in enumerate(val_pairs):
#         inp, tgt = augment(inp, tgt)
#         inp_encoded = tokenizer.encode(inp)
#         tgt_ids = tokenizer.encode(tgt)
#         val_input_ids[idx, :len(inp_encoded)] = inp_encoded
#         val_target_ids[idx, :len(tgt_ids)] = tgt_ids

#     train_data = TensorDataset(torch.LongTensor(train_input_ids).to(device),
#                                torch.LongTensor(train_target_ids).to(device))
#     val_data = TensorDataset(torch.LongTensor(val_input_ids).to(device),
#                                torch.LongTensor(val_target_ids).to(device))

#     train_sampler = RandomSampler(train_data)
#     train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
#     val_sampler = RandomSampler(val_data)
#     val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)
    
#     return train_dataloader, val_dataloader

In [ ]:
keywords = ["/u shopping", "/u todolist", "/u wiki", "/u volume", "/a/"]
filenames = ["shopping_items", "todo_list_items", "wiki_queries", "volumes", "apps"]
augments = {"shopping_items": [], "todo_list_items": [],
            "wiki_queries": [], "volumes": [], "apps": []}

for keyword, filename in zip(keywords, filenames):
    with open(f"/media/nathanmon/389E28739E282BB6/Users/Natha/Datasets/MyJarvisConversation/{filename}.txt", "r") as f:
        for line in f.readlines():
            augments[filename].append(line.replace("\n", "").strip())

In [ ]:
class TokenizedDataset(Dataset):
    def __init__(self, pairs, augments,
                 batch_size=16, max_length=150):
        self.pairs = pairs
        self.augments = augments
        self.batch_size = batch_size
        self.max_length = max_length
        
        self.numbers = ["zero", "one", "two", "three", "four", 
           "five", "six", "seven", "eight", "nine",
           "ten", "eleven", "twelve", "thirteen",
           "fourteen", "fifteen", "sixteen", 
           "seventeen", "eighteen", "nineteen",
           "twenty", "thirty", "forty", "fifty",
           "sixty", "seventy", "eighty", "ninety",
           "hundred", "thousand", "million", "billion",
           "trillion", "quadrillion", "quintillion", "mute", "?"]
        
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def sentence2num(self, sentence):
        words = sentence.split(" ")
        filtered = []
        for word in words:
            if word.lower() in self.numbers:
                filtered.append(word)

        return " ".join(filtered)
    
    def find_tgt(self, response, loc="'"):
        lower = response.index(loc) + len(loc)
        upper = response[lower:].index(loc) + lower
        return response[lower:upper]
    
    def augment(self, inp, tgt):
        keywords = ["/u shopping", "/u todolist", "/u wiki", "/u volume", "/a"]
        filenames = ["shopping_items", "todo_list_items", "wiki_queries", "volumes", "apps"]

        for keyword, filename in zip(keywords, filenames):
            if keyword in tgt or keyword in inp:
                if keyword == "/u volume":
                    prev_item = self.sentence2num(self.find_tgt(tgt))
                elif keyword == "/a":
                    prev_item = self.find_tgt(inp, "/a")
                else:
                    prev_item = self.find_tgt(tgt)
                
                if keyword != "/uvolume" or (prev_item != "?"and prev_item.lower() != "mute"):
                    replacement = random.choice(self.augments[filename])
                    inp = inp.replace(prev_item, replacement)
                    if keyword == "/a":
                        prev_item = self.find_tgt(tgt)
                        
                    tgt = tgt.replace(prev_item, replacement)
                    return inp.replace("/a", ""), tgt.replace("/a", "")
        return inp, tgt

    def getitem(self, idx, augment=False):
        inps_tokenized, inps_types, inps_masked, targs_in, targs_out = [], [], [], [], []
        start_idx = idx*self.batch_size
        for (inp, tgt) in self.pairs[start_idx:start_idx+batch_size]:
            if augment:
                inp, tgt = self.augment(inp, tgt)
            
            inp_encoded = tokenizer.encode_plus(inp,
                                                max_length=MAX_LENGTH,
                                                pad_to_max_length=True)['input_ids']
            tgt_encoded = tokenizer.encode_plus(tgt,
                                                max_length=MAX_LENGTH,
                                                pad_to_max_length=True)['input_ids']
            
            targs_in.append(inp_encoded)
            targs_out.append(tgt_encoded)
            
        targs_in = torch.tensor(targs_in).to(device)
        targs_out = torch.tensor(targs_out).to(device)
        
        return targs_in, targs_out

In [ ]:
pairs = prepareData()

In [ ]:
dataset = TokenizedDataset(pairs, augments, batch_size=16)

In [ ]:
train_ds, val_ds = get_dataloader(16)

# Create Model

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [ ]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion, train=True):
    global batch_size
    total_loss = 0
#     for data in dataloader:
#         input_tensor, target_tensor = data
    for batch in range(len(dataloader) // batch_size):
        input_tensor, target_tensor = dataloader.getitem(batch, augment=True)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        if train:
            loss.backward()

            encoder_optimizer.step()
            decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def train(train_dataloader, val_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_train_losses = []
    print_train_loss_total = 0  # Reset every print_every
    plot_train_loss_total = 0  # Reset every plot_every
    
    print_val_loss_total = 0  # Reset every print_every
    plot_val_loss_total = 0
    plot_val_losses = []

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        train_loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_train_loss_total += train_loss
        plot_train_loss_total += train_loss
        
        # Evaluate validation dataloader
        val_loss = train_epoch(val_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, train=False)
        print_val_loss_total += val_loss
        plot_val_loss_total += val_loss

        if epoch % print_every == 0:
            print_train_loss_avg = print_train_loss_total / print_every
            print_train_loss_total = 0
            print_val_loss_avg = print_val_loss_total / print_every
            print_val_loss_total = 0
            print('%s (%d %d%%) %.4f %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_train_loss_avg, print_val_loss_avg))

        if epoch % plot_every == 0:
            plot_train_loss_avg = plot_train_loss_total / plot_every
            plot_train_losses.append(plot_train_loss_avg)
            plot_train_loss_total = 0
            plot_val_loss_avg = plot_val_loss_total / plot_every
            plot_val_losses.append(plot_val_loss_avg)
            plot_val_loss_total = 0

    showPlot(plot_train_losses, "loss", plot_val_losses, "val_loss")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points, points_name, points2=None, points2_name=None):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.5)
    ax.yaxis.set_major_locator(loc)
    if points2 != None:
        plt.plot(np.arange(len(points)), points, points2)
        plt.legend([points_name, points2_name])
    else:
        plt.plot(points)
        plt.legend([points_name])

In [ ]:
def evaluate(encoder, decoder, sentence):
    with torch.no_grad():
        input_tensor = torch.tensor(tokenizer.encode(sentence), dtype=torch.long).to("cuda").view(1, -1)
        
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(tokenizer.convert_ids_to_tokens(idx.item()))
    return decoded_words, decoder_attn

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 512
batch_size = 16
dropout_p = 0.7

# train_dataloader, val_dataloader = get_dataloader(batch_size)
train_dataloader = TokenizedDataset(pairs[:int(.9 * len(pairs))], augments, 
                                   batch_size=batch_size)
val_dataloader = TokenizedDataset(pairs[int(.9 * len(pairs)):], augments, 
                                   batch_size=batch_size)

encoder = EncoderRNN(len(tokenizer.get_vocab()), hidden_size, dropout_p=dropout_p).to(device)
decoder = AttnDecoderRNN(hidden_size, len(tokenizer.get_vocab()), dropout_p=dropout_p).to(device)

train(train_dataloader, val_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

In [ ]:
torch.save(encoder, "encoder.pth")
torch.save(decoder, "decoder.pth")

In [ ]:
ones_place = {"zero": 0, "one": 1, "two": 2, "three": 3, "four": 4, 
              "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9,
              "ten": 10, "eleven": 11, "twelve": 12, "thirteen": 13,
              "fourteen": 14, "fifteen": 15, "sixteen": 16, 
              "seventeen": 17, "eighteen": 18, "nineteen": 19}

tens_place = {"twenty": 20, "thirty": 30, "forty": 40, "fifty": 50,
              "sixty": 60, "seventy": 70, "eighty": 80, "ninety": 90}

hundreds_place = {"hundred": 10**2, "thousand": 10**3, "million": 10**6, "billion": 10**9,
                  "trillion": 10**12, "quadrillion": 10**15, "quintillion": 10**18}

# Convert word form to numbers
# Works for numbers up to the quintillions
# Also works for dates like nineteen sixty three of twenty twenty four
def word2num(word):
    digits = word.split()
    num = 0
    
    has_ones = False
    has_tens = False
    tens = 0
    hundreds = 0
    for i in range(len(digits)):
        digit = digits[i]
        
        if digit in ones_place:
            # In a real number, there would never be a ones place or tens place next to another ones place
            # For example, there is never nine nineteen or four ninety
            # However, this is seen in a year such as nineteen forty one
            if (has_tens and ones_place[digit] >= 10) or has_ones:
                num *= 100
                
            num += ones_place[digit]
            # Store the tens place and hundreds place in case there is a thousand or million in front of them
            tens += ones_place[digit]
            hundreds += ones_place[digit]
            # Store boolean values to see if there has been a tens place or ones place
            has_ones = True
            has_tens = False
    
        elif digit in tens_place:
            if has_ones or has_tens:
                # If someone enters a year(eighteen twelve works different than one thousand eight hundred twelve)
                num *= 100
                has_tens = False
            else:
                has_tens = True
            
            num += tens_place[digit]
            tens += tens_place[digit]
            hundreds += tens_place[digit]
            has_ones = False
            
        elif digit in hundreds_place:
            if digit != "hundred" and hundreds:
                num += hundreds * hundreds_place[digit] - hundreds
                hundreds = 0
                tens = 0
            else:
                num += tens * hundreds_place[digit] - tens
                hundreds = tens * hundreds_place[digit]
                tens = 0
            has_ones = False
            has_tens = False
        
    return num

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
}
def weather(city, wob):
    res = requests.get(
        f'https://www.google.com/search?q={city}+weather&rlz=1C1VDKB_enUS1016US1016&oq={city}&aqs=chrome.0.69i59j69i57j69i59l2j0i271l2j69i61l2.905j0j7&sourceid=chrome&ie=UTF-8',
        headers=headers
    )
    soup = bs4(res.text, 'html.parser')
    return soup.select(f"#wob_{wob}")[0].getText().strip()

def volume(vol):
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))

    volume.SetMasterVolumeLevelScalar(word2num(vol)/100, None)

In [ ]:
while True:
    chat = input("Speak: ")
    output = evaluate(encoder, decoder, chat)
    response = ' '.join(output[0][:-1])
    
    print("Response: ", response)
    # Get the date
    if "/udate" in response:
        response = response.replace("/udate", datetime.date.today().strftime("%B %d, %Y"))
    # Get the time
    if "/utime" in response:
        response = response.replace("/utime", datetime.datetime.now().strftime("%I:%M:%S"))
    # Get the temperature
    if "/utemp" in response:
        response = response.replace("/utemp", weather("boston", "tm"))
    # Get the humidity
    if "/uhumidity" in response:
        response = response.replace("/uhumidity", weather("boston", "hm"))
    # Get the wind speed
    if "/uwind" in response:
        response = response.replace("/uwind", weather("boston", "ws"))
    # Get the amount of precipitation
    if "/uprecipitation" in response:
        response = response.replace("/uprecipitation", weather("boston", "pp"))
    if "/uvolume" in response:
        after = response.split("/uvolume")[-1]
        vol = after.split("'")[1]
        #response = response.replace("/uvolume"+"'"+vol+"'", "")
        volume(vol)
    if "/usleep" in response:
        response = response.replace("/usleep", "")
        print(response)
        break
    if "/unewtab" in response:
        response = response.replace("/unewtab", "")
        pyautogui.hotkey('ctrl', 't')
    if "/uclosetab" in response:
        response = response.replace("/uclosetab", "")
        pyautogui.hotkey('ctrl', 'w')
    if "/uswitchtab" in response:
        after = response.split("/uvolume")[-1]
        new = after.split("'")[1]
        response = response.replace("/uswitchtab"+"'"+new+"'", "")
        pyautogui.hotkey('ctrl', str(word2num(new)))
    
    print("Filtered: ", response)